In [ ]:

%env SPARK_HOME = /home/conorstreete/CS4337/lib/python3.8/site-packages/pyspark


env: SPARK_HOME=/home/conorstreete/CS4337/lib/python3.8/site-packages/pyspark


**BEFORE RUNNING CODE PLEASE ENSURE YOU HAVE THE LATEST PULL FROM MASTER OF COVID 19 DATASET**

**ATTENTION ! ! ! ! !**




In [ ]:
#Importing all required functions from libraries I will be using

import findspark
import pyspark
import pyspark.sql.functions as f
from pyspark.sql.types import *
import pyspark.sql.types
from pyspark.sql import SparkSession
import pandas_bokeh
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from pyspark.sql.functions import col, asc
from pyspark.sql.functions import lit

from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
from bokeh.models.tools import HoverTool
from datetime import timedelta, date,datetime
from bokeh.models import Label
from bokeh.models import BoxAnnotation
!pip install a-world-of-countries

# Import the AWOC package.
import awoc

# Initialize the AWOC class.
my_world = awoc.AWOC()



findspark.init()

You should consider upgrading via the '/home/conorstreete/CS4337/bin/python -m pip install --upgrade pip' command.


In [ ]:
#Priting working directorty to help location of files
import os
cwd = os.getcwd()
print(cwd)

/home/conorstreete


In [ ]:
#!/usr/bin/env python
#Initilizing spark enviroment
import sys

# set up environment
# Build the SparkSession
spark = SparkSession.builder \
  .master("local") \
  .appName("Project 2 CS4337") \
  .config("spark.executor.memory", "3gb") \
  .getOrCreate()

sc = spark.sparkContext
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [ ]:
#This method takes in a dataframe and time element 
def getAllConfirmed(current,date):

  #selects only the names coloumns that I want to use
  newC = current[["Country_Region","Confirmed","Deaths","Active"]]
  #adds all rows together which have the same country name
  newC = newC.groupby(newC['Country_Region'],as_index = False).sum()
  #Creats Null column for the calualtion of todays cases
  newC['Todays_Cases'] = np.NaN
  #sets the date to the date of the dataframe data inside
  currDate = datetime.strptime(date,"%m-%d-%Y")
  currDate = currDate.strftime("%Y-%m-%d")
  newC['Date'] = currDate

  return newC

In [ ]:
casesDirectory = (r"/home/conorstreete//CS4337/COVID-19/csse_covid_19_data/"
"csse_covid_19_daily_reports/")
dailyCases = {} 

# imports all the files from my directory and then adds them to a 
# dictionary of {date:Dataframe}

for file in os.listdir(casesDirectory):
    fullPath = os.path.join(casesDirectory, file)
    try:
      date = re.search('(.+?).csv', file).group(1)
      currentCase = pd.read_csv(fullPath, index_col=None, header=0)
      dailyCases[date] = currentCase
      print(date)
    except Exception:
      print("File name not compatible type")


08-06-2020
11-23-2020
10-16-2020
09-25-2020
09-19-2020
02-10-2020
07-02-2020
09-30-2020
11-21-2020
06-16-2020
04-17-2020
08-01-2020
04-07-2020
02-08-2020
09-26-2020
03-25-2020
06-07-2020
09-07-2020
12-15-2020
05-30-2020
07-18-2020
06-01-2020
08-18-2020
11-14-2020
04-29-2020
08-26-2020
06-03-2020
12-05-2020
03-27-2020
06-30-2020
05-19-2020
05-21-2020
02-17-2020
File name not compatible type
02-13-2020
03-06-2020
02-26-2020
06-25-2020
05-15-2020
05-27-2020
10-11-2020
10-26-2020
02-24-2020
03-14-2020
04-18-2020
08-05-2020
06-10-2020
03-18-2020
07-12-2020
07-24-2020
06-24-2020
10-13-2020
09-04-2020
02-16-2020
10-04-2020
06-11-2020
01-30-2020
09-11-2020
09-18-2020
04-21-2020
10-27-2020
02-19-2020
05-10-2020
02-03-2020
10-17-2020
08-30-2020
03-17-2020
File name not compatible type
05-02-2020
02-23-2020
09-01-2020
09-10-2020
06-20-2020
10-08-2020
08-22-2020
05-24-2020
03-20-2020
12-06-2020
08-14-2020
09-02-2020
02-29-2020
12-02-2020
04-11-2020
02-22-2020
02-27-2020
08-03-2020
04-02-2020
08-28

In [ ]:
from collections import OrderedDict
#Sorts the dictionary by date from smallest to largest
allSortedDailyCases = OrderedDict(sorted(dailyCases.items()))
print(allSortedDailyCases)

OrderedDict([('01-22-2020',     Province/State  Country/Region      Last Update  Confirmed  Deaths  \
0            Anhui  Mainland China  1/22/2020 17:00        1.0     NaN   
1          Beijing  Mainland China  1/22/2020 17:00       14.0     NaN   
2        Chongqing  Mainland China  1/22/2020 17:00        6.0     NaN   
3           Fujian  Mainland China  1/22/2020 17:00        1.0     NaN   
4            Gansu  Mainland China  1/22/2020 17:00        NaN     NaN   
5        Guangdong  Mainland China  1/22/2020 17:00       26.0     NaN   
6          Guangxi  Mainland China  1/22/2020 17:00        2.0     NaN   
7          Guizhou  Mainland China  1/22/2020 17:00        1.0     NaN   
8           Hainan  Mainland China  1/22/2020 17:00        4.0     NaN   
9            Hebei  Mainland China  1/22/2020 17:00        1.0     NaN   
10    Heilongjiang  Mainland China  1/22/2020 17:00        NaN     NaN   
11           Henan  Mainland China  1/22/2020 17:00        5.0     NaN   
12       H

In [ ]:
from datetime import timedelta, date,datetime
#Gets the range of dates between two dates
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)
 

In [ ]:
#creating structure for Pyspark dataframe
schema = StructType([StructField('Country_Region',StringType(),True),
                     StructField('Confirmed',IntegerType(),True),
                     StructField('Deaths',IntegerType(),True),
                     StructField('Active',IntegerType(),True),
                     StructField('Todays_Cases',FloatType(),False),
                     StructField('Date',StringType(),False)
                     ])

In [ ]:
#This method return a dataframe which contains the data for a specfic country 
#between dates
def filterLocation(location,selectDailyCases):

  todayDF = ""
  ydayDF = ""
  newDailyCases = {}
  daily = []
  numberOfDays = 60
  totalDaily = 0 
  allTogetherP = pd.DataFrame()

  #scan through each dataframe of the selected dates

  for key,value in selectDailyCases.items():
  
    if(ydayDF != ""):

      numberOfDays = numberOfDays + 1
      today = spark.createDataFrame(value, schema)

      #creates new data frame of the Confirmed cases from yday

      ydayConf = ydayDF.select(col('Country_Region'), col('Confirmed').alias('yConf'))

      #adds it to todays cases
      
      df = today.join(ydayConf, on = ['Country_Region'], how = 'inner')
      #Takes total confirmed yday from total confirmed today to calulate 
      #the cases of each day
      tdC = df.select(col('Confirmed') - col('yConf'))

      # adds a new column with these figures
      new = df.withColumn('Todays_Cases',f.abs(col('Confirmed') - col('yConf')))
      #removes the yConf coloumn as no longer needed
      new = new.drop(col('yConf'))

      if(location != ""):
        #filters so only data where the country is equal to the selected name
        new = new.where(col('Country_Region') == location)
      
      #appends all the dataframes together so that the data is grouped as one
      new = new.toPandas()
      allTogetherP = allTogetherP.append(new,ignore_index = True)

      
      newDailyCases[key] = new
      
    value = spark.createDataFrame(value, schema)
    today = value
    ydayDF = today

  #returns the dataframe of dates for the location selecteed
  sortedDailyCases = OrderedDict(sorted(newDailyCases.items()))
  
  return allTogetherP

In [ ]:

#returns all the dataframes for data between selected dates
def chooseDateRange(start_date,end_date):
  selectDailyCases = {}

  #scans through each date needed
  for single_date in daterange(start_date, end_date):
    #formats date for graphs later on
    currDate = single_date.strftime("%m-%d-%Y")
    #gets the data for the current date
    data = allSortedDailyCases.get(currDate)
    data = pd.DataFrame(data)
    print(data.columns)
    print(currDate)
    #passes selected data to the getAllConfirmed for processing 
    data = getAllConfirmed(data,currDate)

    selectDailyCases[currDate]  =  data
  
  #retuns a dictionary of data for inbetween selected dates
  return selectDailyCases
    

In [ ]:
start_date = date(2020, 10, 1)
end_date = date(2020, 12, 1)

#selecting data rang for 2 months in Ireland
selectedDates = chooseDateRange(start_date,end_date)
irelandCases = filterLocation('Ireland',selectedDates)



Index(['FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Last_Update',
       'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'Combined_Key', 'Incidence_Rate', 'Case-Fatality_Ratio'],
      dtype='object')
10-01-2020
Index(['FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Last_Update',
       'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'Combined_Key', 'Incidence_Rate', 'Case-Fatality_Ratio'],
      dtype='object')
10-02-2020
Index(['FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Last_Update',
       'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'Combined_Key', 'Incidence_Rate', 'Case-Fatality_Ratio'],
      dtype='object')
10-03-2020
Index(['FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Last_Update',
       'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'Combined_Key', 'Incidence_Rate', 'Case-Fatality_Ratio'],
      dtype='object')
10-04-2020
Index(['FIPS', 'Admin2', 'Pr

In [ ]:
#Printing to ensure data is what we asked for 
print(irelandCases)

   Country_Region  Confirmed  Deaths  Active  Todays_Cases        Date
0         Ireland      37063    1801   11898           466  2020-10-02
1         Ireland      37668    1810   12494           605  2020-10-03
2         Ireland      38032    1810   12858           364  2020-10-04
3         Ireland      38549    1810   13375           517  2020-10-05
4         Ireland      38973    1811   13798           424  2020-10-06
5         Ireland      39584    1816   14404           611  2020-10-07
6         Ireland      40086    1817   14905           502  2020-10-08
7         Ireland      40703    1821   15518           617  2020-10-09
8         Ireland      41714    1824   16526          1011  2020-10-10
9         Ireland      42528    1826   17338           814  2020-10-11
10        Ireland      43351    1827   18160           823  2020-10-12
11        Ireland      44159    1830   18965           808  2020-10-13
12        Ireland      45243    1835   20044          1084  2020-10-14
13    

In [ ]:
#Converts data to pandas for bokeh
irelandCases['Date']= pd.to_datetime(irelandCases['Date'])
#Adds a new column containing the 5 day rolling average of the daily cases
irelandCases['MA'] = irelandCases.iloc[:,4].rolling(window = 5).mean()

**Graph 1 - Examining the effect of a lockdown period on confirmed daily cases**

In [ ]:
from bokeh.models import Legend, LegendItem

ie = ColumnDataSource(irelandCases)




#outputs graphs to notebook
output_notebook()

#setting plot details
fig = figure(x_axis_type = 'datetime',
             plot_height  =600, plot_width = 600,
             title='Graph 1 : Ireland Cases Over A 2 Month Period With Lockdown - INTERACTIVE',
             x_axis_label= 'Virus Period', y_axis_label = 'Confirmed Cases',
             toolbar_location=None)

#plots the daily cases
fig.circle(x = 'Date', y = 'Todays_Cases', source = ie, size = 10,
           color = 'green',legend_label = "Todays Cases")

fig.legend.location = 'top_left'

#plots the moving average
movingA = fig.line(x = 'Date', y = 'MA', 
         color='red', legend_label = '5 Day AVG', 
         line_width = 1.90,
         source = ie)

#Creates a box to show lockdown period

box_left = pd.to_datetime('21-10-2020')
box_right = pd.to_datetime('12-01-2020')

box = BoxAnnotation(left = box_left, right = box_right,
                    line_width = 1, line_color = 'black', line_dash = 'dashed',
                    fill_alpha = 0.2, fill_color = 'orange')

fig.add_layout(box)


#adding label to give user more information

mytext = Label(x = 275, y = 350, x_units = 'screen', y_units = 'screen',
                 text = 'Lockdown Period of 6Wks', render_mode = 'css',
                 border_line_color = 'black', border_line_alpha = 1.0,
                 background_fill_color = 'white', background_fill_alpha = 1.0)
fig.add_layout(mytext)

fig.xgrid.visible = False
fig.ygrid.visible = False
fig.toolbar.active_drag = None


#Adding graph interaction using a hover tool
hover = HoverTool(
    tooltips = [("Todays Cases", "@Todays_Cases"),
                ("5 Day Moving AVG","@MA{0000}"),
                ("Date","@Date{%F}")],

    formatters = {
        '@Date': 'datetime'
    },
    mode = 'vline',
    renderers = [movingA])

fig.add_tools(hover)
fig.yaxis.minor_tick_line_color = None
fig.outline_line_color = None


show(fig)

Loading BokehJS ...

In [ ]:
#Similar method to filter location but gets all the dates data for a given country
def getAllDates(country):
  allD = pd.DataFrame()

  for key,value in allSortedDailyCases.items():
    dfa = pd.DataFrame(value)

    #setting date 
    currDate = datetime.strptime(key,"%m-%d-%Y")
    currDate = currDate.strftime("%Y-%m-%d")
   
    try:
      dff = dfa[["Country_Region","Confirmed","Deaths","Active"]]
      dff = dff.groupby(dff['Country_Region'],as_index=False).sum()
      
      new = dff[dff["Country_Region"] == country]
      df2 = new.assign(Date = currDate)
      print(df2)

      #adds current date of data
      df2['Date'] = pd.to_datetime(df2['Date'])
     
  
      allD = allD.append(df2,ignore_index = True)
    except:
      print()


  return allD


In [ ]:
#Gets all available data for the countries selected
allIreland = getAllDates("Ireland")
allGermany = getAllDates("Germany")
allPortugal = getAllDates("Portugal")
allFranch = getAllDates("France")
allSpain = getAllDates("Spain")

**Graph 2 - Comparing 5 European countries confirmed cases over the the pandemic period:**

In [ ]:
ie = ColumnDataSource(allIreland)
ger = ColumnDataSource(allGermany)
port = ColumnDataSource(allPortugal)
fran = ColumnDataSource(allFranch)
spain = ColumnDataSource(allSpain)

# Create the checkbox selection element, available carriers is a  
# list of all airlines in the data

output_notebook()

fig = figure(x_axis_type = 'datetime',
             plot_height = 600, plot_width = 600,
             title='Graph 2 - Select Country Cases over time - INTERACTIVE',
             x_axis_label='Date', y_axis_label='Confirmed Cases',
             toolbar_location = None)

#Plotting the data for each of the selected countires

fig.line(x = 'Date', y = 'Confirmed', 
         color='red', legend_label='Ireland', 
         line_width=1.90,
         source=ie)

fig.line(x = 'Date', y = 'Confirmed', 
         color='blue', legend_label='Germany', 
         line_width=1.90,
         source=ger)


fig.line(x = 'Date', y = 'Confirmed', 
         color='green', legend_label='Portugal', 
         line_width=1.90,
         source=port)

fig.line(x = 'Date', y = 'Confirmed', 
         color='black', legend_label='France', 
         line_width=1.90,
         source=fran)

fig.line(x = 'Date', y = 'Confirmed', 
         color='orange', legend_label='Spain', 
         line_width=1.90,
         source=spain)

hover = HoverTool(
    
    tooltips=[("Total Confirmed", "@Confirmed"),
                ("Date","@Date{%F}"),
              ("Country","@Country_Region")],

    formatters = {
      '@Date': 'datetime'
    },
    )

fig.add_tools(hover)
fig.yaxis.formatter=NumeralTickFormatter(format="0,000000")

mytext = Label(x = 20, y=350, x_units = 'screen', y_units='screen',
                 text = 'Click A Country', render_mode='css',
                 border_line_color = 'black', border_line_alpha=1.0,
                 background_fill_color = 'white', background_fill_alpha = 1.0)
fig.add_layout(mytext)



#sets interaction to the diagram allowing the data to be added or removed from the graph
fig.legend.location = "top_left"
fig.legend.click_policy="hide"

fig.yaxis.minor_tick_line_color = None
fig.outline_line_color = None

fig.xgrid.visible = False
fig.ygrid.visible = False
fig.toolbar.active_drag = None

show(fig)

Loading BokehJS ...

**The above graph is an example of an interactive graph. A user can hover over the countires line to view specific details along the axis and they can also click the name of the country in the legend to remove or at it from ther graph**

In [ ]:
allAus = getAllDates("Austria")
allItaly = getAllDates("Italy")
allPoland = getAllDates("Poland")
allHungary = getAllDates("Hungary")
allSlovakia = getAllDates("Slovakia")

**Graph 3 - Death Cases and Spike detection in several European countries**

In [ ]:
aus = ColumnDataSource(allAus)
it = ColumnDataSource(allItaly)
pol = ColumnDataSource(allPoland)
hung = ColumnDataSource(allHungary)
slov = ColumnDataSource(allSlovakia)

from bokeh.models import Band

#The following code below creates seperate graphs and shows them all together to compare the deaths in the countries
output_notebook()

p = figure(x_axis_type='datetime',plot_height = 300, plot_width = 300,x_axis_label = 'Virus Period',
                  y_axis_label = 'Deaths',
                  toolbar_location = None,y_range  = (0,25000))
p.line(x  =  "Date",y = "Deaths",color = 'red', 
         line_width = 1.90,
         source = aus)
p.title.text  = "Austria"
p.yaxis.formatter = NumeralTickFormatter(format = "0,000000")
band = Band(base = 'Date', upper = 'Deaths', source=aus, level='underlay',
            fill_alpha = 1, fill_color = '#FF9484')
p.add_layout(band)


p1 = figure(x_axis_type = 'datetime',plot_height = 300, plot_width = 300,x_axis_label = 'Virus Period',
                  y_axis_label = 'Deaths',
                  toolbar_location = None,y_range = (0,75000))
p1.line(x = "Date",y = "Deaths",color = 'red', 
         line_width = 1.90,
         source = it)
p1.title.text = "Italy"
p1.yaxis.formatter = NumeralTickFormatter(format = "0,000000")
band1  =  Band(base = 'Date', upper = 'Deaths', source = it, level = 'underlay',
            fill_alpha = 1, fill_color = '#FF9484')
p1.add_layout(band1)

p2 = figure(x_axis_type = 'datetime',plot_height = 300,
                  plot_width = 300,x_axis_label = 'Virus Period',
                  y_axis_label = 'Deaths',
                  toolbar_location = None,y_range  = (0,25000))
p2.line(x = "Date",y = "Deaths",color='red', 
          line_width = 1.90,
          source=pol,
          line_alpha  = 1.0)
p2.title.text = "Poland"
p2.yaxis.formatter = NumeralTickFormatter(format = "0,000000")
band2 = Band(base = 'Date', upper = 'Deaths', source = pol, level = 'underlay',
            fill_alpha = 1, fill_color = '#FF9484')
p2.add_layout(band2)

p3 = figure(x_axis_type = 'datetime',plot_height = 300, 
            plot_width = 300,x_axis_label = 'Virus Period',
                  y_axis_label='Deaths',
                  toolbar_location = None,y_range = (0,25000))
p3.line(x = "Date",y = "Deaths",color = 'red', 
         line_width = 1.90,
         source = hung)
p3.title.text = "Hungary"
p3.yaxis.formatter = NumeralTickFormatter(format = "0,000000")
band3 = Band(base = 'Date', upper = 'Deaths', source = hung, level = 'underlay',
            fill_alpha = 1, fill_color = '#FF9484')
p3.add_layout(band3)



p5 = figure(x_axis_type='datetime',plot_height=300, plot_width=300,x_axis_label='Virus Period',
                  y_axis_label='Deaths',
                  toolbar_location=None,y_range =(0,25000))
p5.line(x = "Date",y="Deaths",color='red', 
         line_width=1.90,
         source=ie)
p5.title.text = "Ireland"
p5.yaxis.formatter=NumeralTickFormatter(format="0,000000")
band5 = Band(base='Date', upper='Deaths', source=ie, level='underlay',
            fill_alpha=1, fill_color='#FF9484')
p5.add_layout(band5)


p6 = figure(x_axis_type='datetime',plot_height=300, plot_width=300,x_axis_label='Virus Period',
                  y_axis_label='Deaths',
                  toolbar_location=None,y_range =(0,75000))
p6.line(x = "Date",y="Deaths",color='red', 
         line_width=1.90,
         source=spain)
p6.title.text = "Spain"
p6.yaxis.formatter=NumeralTickFormatter(format="0,000000")
band6 = Band(base='Date', upper='Deaths', source=spain, level='underlay',
            fill_alpha=1, fill_color='#FF9484')
p6.add_layout(band6)


p7 = figure(x_axis_type='datetime',plot_height=300, plot_width=300,x_axis_label='Virus Period',
                  y_axis_label='Deaths',
                  toolbar_location=None,y_range =(0,75000))
p7.line(x = "Date",y="Deaths",color='red', 
         line_width=1.90,
         source=fran)
p7.title.text = "France"
p7.yaxis.formatter=NumeralTickFormatter(format="0,000000")
band7 = Band(base='Date', upper='Deaths', source=fran, level='underlay',
            fill_alpha=1, fill_color='#FF9484')
p7.add_layout(band7)


p8 = figure(x_axis_type='datetime',plot_height=300, plot_width=300,x_axis_label='Virus Period',
                  y_axis_label='Deaths',
                  toolbar_location=None,y_range =(0,25000))
p8.line(x = "Date",y="Deaths",color='red', 
         line_width=1.90,
         source=port)
p8.title.text = "Portugal"
p8.yaxis.formatter=NumeralTickFormatter(format="0,000000")
band8 = Band(base='Date', upper='Deaths', source=port, level='underlay',
            fill_alpha=1, fill_color='#FF9484')
p8.add_layout(band8)

east_west_gridplot = gridplot([[p1,p6,p7],[p5,p2],[p,p3,p8]], 
                              toolbar_location='right')

#Simplifying the graph by removing the grid lines


p.xgrid.visible = False
p.ygrid.visible = False
p1.xgrid.visible = False
p1.ygrid.visible = False
p2.xgrid.visible = False
p2.ygrid.visible = False
p3.xgrid.visible = False
p3.ygrid.visible = False
p5.xgrid.visible = False
p5.ygrid.visible = False
p6.xgrid.visible = False
p6.ygrid.visible = False
p7.xgrid.visible = False
p7.ygrid.visible = False
p8.xgrid.visible = False
p8.ygrid.visible = False


p.yaxis.ticker = [0, 25000, 75000]
p1.yaxis.ticker = [0, 25000, 75000]
p2.yaxis.ticker = [0, 25000, 75000]
p3.yaxis.ticker = [0, 25000, 75000]
p5.yaxis.ticker = [0, 25000, 75000]
p6.yaxis.ticker = [0, 25000, 75000]
p7.yaxis.ticker = [0, 25000, 75000]
p8.yaxis.ticker = [0, 25000, 75000]

p.xaxis.ticker = [0]
p1.xaxis.ticker = [0]
p2.xaxis.ticker = [0]
p3.xaxis.ticker = [0]
p5.xaxis.ticker = [0]
p6.xaxis.ticker = [0]
p7.xaxis.ticker = [0]
p8.xaxis.ticker = [0]

p.outline_line_color = None
p1.outline_line_color = None
p2.outline_line_color = None
p3.outline_line_color = None
p5.outline_line_color = None
p6.outline_line_color = None
p7.outline_line_color = None
p8.outline_line_color = None

p.xaxis.major_tick_line_color = None        # turn off x-axis major ticks
p.xaxis.minor_tick_line_color = None 


show(east_west_gridplot)

Loading BokehJS ...

 **The above graphs area all grouped together to allow for  a comparision in the rate of deaths increasing since the start of covid 19
the top 3 are scaled to 70000 deaths while the bottom 5 are scaled to 25000. This is done to see a clearer picture of the rapid or non rapid growth of cases** 

**For the next code to run the user must make sure they have the latest pull of the data from github** 

In [ ]:
from datetime import date 
from datetime import timedelta 

ydayStats = pd.DataFrame()
#Calculating today and yesterdays date
today = date.today() 
yday = today - timedelta(days = 1) 
yday = yday.strftime('%m-%d-%Y')



In [ ]:
#gets all the latest case statistics
ydayStats = allSortedDailyCases.get(yday)

#gets a lsit of the european countries from a library
countries_of_europe = my_world.get_countries_list_of('Europe')

#Creates a dataframe which matched the countires name with that of the european list
rslt_df = ydayStats[ydayStats['Country_Region'].isin(countries_of_europe)] 

#some preprocceesing similar to earlier code
ydayStats = rslt_df[["Country_Region","Confirmed","Deaths","Active"]]
ydayStats = ydayStats.groupby(ydayStats['Country_Region'],
                              as_index = False).sum()
ydayStats =ydayStats.sort_values(by = 'Active', ascending = False,
                                ignore_index = True)

#gets a list of all the countries that we have selected from the data
countries = ydayStats['Country_Region'].tolist() 
print(countries)

['France', 'United Kingdom', 'Spain', 'Italy', 'Netherlands', 'Belgium', 'Russia', 'Ukraine', 'Germany', 'Sweden', 'Serbia', 'Poland', 'Hungary', 'Greece', 'Bulgaria', 'Romania', 'Switzerland', 'Portugal', 'Lithuania', 'Ireland', 'Slovakia', 'Denmark', 'Austria', 'Bosnia and Herzegovina', 'Norway', 'Albania', 'Croatia', 'Belarus', 'Slovenia', 'Moldova', 'Cyprus', 'Kosovo', 'Montenegro', 'Latvia', 'Finland', 'Luxembourg', 'Estonia', 'Malta', 'Andorra', 'San Marino', 'Liechtenstein', 'Iceland', 'Monaco']


**Graph 4 - All European countries active cases yesterday in order from highest to lowest**

In [ ]:
source = ColumnDataSource(ydayStats)
from bokeh.palettes import Category20,Category10

from bokeh.transform import factor_cmap
output_notebook()

#sets the color pallette of the data based on the countries
pal = Category20[20]
color_map = factor_cmap(field_name = 'Country_Region',
                    palette = pal, factors = countries)

p = figure(plot_width = 800, x_range = countries, 
           title = "Graph 4 - All European countries active cases yesterday"
            "in order from highest to lowest")

#bar chart
p.vbar(  x = 'Country_Region', top = 'Active', source = source, width = 1, 
       color = color_map, line_color = "white")

hover = HoverTool(
    tooltips = [("Total Active", "@Active{0,000,000}"),
              ("Country","@Country_Region")],
    )

p.add_tools(hover)

#formats the y axis data
p.yaxis.formatter = NumeralTickFormatter(format = "0,000000000")

#sets the x axis specifications
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Countries grouped by Active cases for " + yday
p.xaxis.major_label_orientation = 1.2
p.outline_line_color = None
p.xgrid.visible = False
p.ygrid.visible = False

p.yaxis.ticker = [0, 500000, 1000000, 1500000, 2000000]

show(p)

Loading BokehJS ...

/home/conorstreete/CS4337/lib/python3.8/site-packages/bokeh/models/mappers.py:150: UserWarning: Palette length does not match number of factors. ['Slovakia', 'Denmark', 'Austria', 'Bosnia and Herzegovina', 'Norway', 'Albania', 'Croatia', 'Belarus', 'Slovenia', 'Moldova', 'Cyprus', 'Kosovo', 'Montenegro', 'Latvia', 'Finland', 'Luxembourg', 'Estonia', 'Malta', 'Andorra', 'San Marino', 'Liechtenstein', 'Iceland', 'Monaco'] will be assigned to `nan_color` gray
  warnings.warn("Palette length does not match number of factors. %s will be assigned to `nan_color` %s" % (extra_factors, self.nan_color))


**This graph shows from highest to lowest the number of active cases based on european country**